In [10]:
import pandas as pd
import numpy as np
from flaml import AutoML
import yaml
pd.set_option('display.max_columns', None)


%load_ext autoreload
%autoreload 2
%reload_ext autoreload




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# Read YAML file
with open("config.yaml", "r") as stream:
    config_input = yaml.safe_load(stream)

# FLAML
FLAML_SETTINGS = config_input["flaml"]["settings"]

# Data
FEATURE_SET = config_input["data"]["feature_set"]
TEST_YEAR = config_input["data"]["training"]["test_year"]
TARGET = config_input["data"]["training"]["target"]
TRAIN_COLUMNS = config_input["data"]["training"]["train_cols"]


NameError: name 'config_input' is not defined

In [54]:
from dotenv import load_dotenv
import os

load_dotenv()


ROOT_DIRECTORY = os.environ["ROOT_DIRECTORY"]
df = pd.read_csv(ROOT_DIRECTORY + "/data/processed/processed_features.csv")

In [55]:
df.columns

Index(['Unnamed: 0', 'Rnd', 'R', 'Team', 'Opponent', 'Date', 'game_year',
       'game_key', 'round_num', 'T',
       ...
       'away_win_joined', 'winsum_joined', 'games_joined', 'percentage_joined',
       'percentage_wins_joined', 'home_percentage_wins_joined',
       'away_percentage_wins_joined', 'ladder_position_joined',
       'join_year_joined', 'rem_flag'],
      dtype='object', length=132)

In [56]:
df.loc[df['game_year']==2023].sort_values(by='Rnd',ascending=False)

,Unnamed: 0,Rnd,R,Team,Opponent,Date,game_year,game_key,round_num,T,win,season_status,rolling_for,M,rolling_wins,rolling_games,rolling_against,rolling_margin,rolling_percentage,rolling_F_median_1,rolling_A_median_1,rolling_F_sum_1,rolling_A_sum_1,rolling_M_sum_1,rolling_wins_1,rolling_wins_2,rolling_wins_3,rolling_M_1,rolling_M_2,rolling_M_3,prev_match_M_1,rolling_M_median_1,rolling_M_median_2,rolling_M_median_3,prev_match_M_2,prev_match_M_3,prev_match_M_4,prev_match_M_5,prev_match_M_6,prev_match_M_7,prev_match_M_8,prev_match_M_9,prev_match_M_10,prev_match_M_11,rolling_game_points,home_rolling_for,home_rolling_against,rolling_percentage_1,home_for,home_against,early,mid,game_points,F,A,home_game,home_win,away_game,away_win,winsum,games,percentage,percentage_wins,home_percentage_wins,away_percentage_wins,ladder_position,join_year,Rnd_joined,R_joined,Team_joined,Opponent_joined,game_year_joined,game_key_joined,T_joined,win_joined,season_status_joined,rolling_for_joined,M_joined,rolling_wins_joined,rolling_games_joined,rolling_against_joined,rolling_margin_joined,rolling_percentage_joined,rolling_F_median_1_joined,rolling_A_median_1_joined,rolling_F_sum_1_joined,rolling_A_sum_1_joined,rolling_M_sum_1_joined,rolling_wins_1_joined,rolling_wins_2_joined,rolling_wins_3_joined,rolling_M_1_joined,rolling_M_2_joined,rolling_M_3_joined,prev_match_M_1_joined,rolling_M_median_1_joined,rolling_M_median_2_joined,rolling_M_median_3_joined,prev_match_M_2_joined,prev_match_M_3_joined,prev_match_M_4_joined,prev_match_M_5_joined,prev_match_M_6_joined,prev_match_M_7_joined,prev_match_M_8_joined,prev_match_M_9_joined,prev_match_M_10_joined,prev_match_M_11_joined,rolling_game_points_joined,home_rolling_for_joined,home_rolling_against_joined,rolling_percentage_1_joined,home_for_joined,home_against_joined,early_joined,mid_joined,game_points_joined,F_joined,A_joined,home_game_joined,home_win_joined,away_game_joined,away_win_joined,winsum_joined,games_joined,percentage_joined,percentage_wins_joined,home_percentage_wins_joined,away_percentage_wins_joined,ladder_position_joined,join_year_joined,rem_flag
5738,5906,R7,W,Western Bulldogs,Hawthorn,2023-04-29 16:35:00,2023,40119.02023-04-29 16:35:007,7,H,1,mid,437,29,3,7,484,-47,0.902893,66.5,77.0,729.0,793.0,-64.0,16.0,12.0,20.0,-24.0,-96.0,200.0,49.0,-4.5,5.0,10.0,-14.0,5.0,14.0,-51.0,-50.0,23.0,5.0,-17.0,-28.0,10.0,12,109.0,146,0.902692,94,65,0,1,48,1973,1812,11,7,11,5,12,22,1.088852,0.545455,0.636364,0.454545,8,2023,R7,L,Hawthorn,Western Bulldogs,2023,40119.02023-04-29 16:35:007,A,0,mid,379,-29,1,7,587,-208,0.645656,66.0,83.0,642.0,939.0,-297.0,4.0,16.0,4.0,-231.0,15.0,-171.0,-3.0,-17.5,-12.0,-4.0,-2.0,-82.0,19.0,-81.0,-59.0,-23.0,-61.0,7.0,-12.0,46.0,4,222.0,265,0.645051,0,0,0,1,32,1787,1991,11,6,11,2,8,22,0.897539,0.363636,0.545455,0.181818,13,2023,NaN
5363,5521,R7,L,West Coast,Carlton,2023-04-29 17:30:00,2023,47940.02023-04-29 17:30:007,7,H,0,mid,471,-108,1,7,648,-177,0.726852,75.5,107.5,753.0,1058.0,-305.0,4.0,0.0,4.0,-262.0,-194.0,-341.0,-40.0,-32.0,-24.0,-28.0,-47.0,-63.0,-41.0,19.0,-5.0,-85.0,-24.0,-16.0,-3.0,-28.0,4,164.0,208,0.726430,44,152,0,1,8,1429,2389,11,1,11,1,2,22,0.598158,0.090909,0.090909,0.090909,17,2023,R7,W,Carlton,West Coast,2023,47940.02023-04-29 17:30:007,A,1,mid,452,108,3,7,489,-37,0.924335,74.0,82.0,735.0,840.0,-105.0,14.0,8.0,20.0,-38.0,-13.0,131.0,-22.0,-3.0,-5.0,26.0,-56.0,23.0,10.0,8.0,0.0,-1.0,-5.0,-33.0,-29.0,36.0,14,151.0,165,0.924180,0,0,0,1,48,1857,1714,11,8,11,4,12,22,1.083431,0.545455,0.727273,0.363636,9,2023,NaN
307,315,R7,L,Adelaide,Collingwood,2023-04-30 16:10:00,2023,43942.02023-04-30 16:10:007,7,H,0,mid,592,-1,4,7,511,81,1.158513,96.0,81.0,935.0,836.0,99.0,16.0,12.0,8.0,25.0,-25.0,-102.0,3.0,22.5,16.0,-29.0,56.0,39.0,31.0,-32.0,-16.0,-56.0,29.0,16.0,29.0,-33.0,16,306.0,243,1.158824,58,59,0,1,32,1721,1986,11,5,11,3,8,22,0.866566,0.363636,0.454545,0.272727,14,2023,R7,W,Collingwood,Adelaide,2023,43942.02023-04-30 16:10:007,A,1,mid,567,1,5,7,474,93,1.196203,85.5,77.0,8

In [16]:
df = pd.read_csv(FEATURE_SET)
df = df.loc[df['game_year']>=1995]
# df["Date"] = pd.to_datetime(df["Date"])
# df = df.loc[df['round_num']<=10]
df_test = df.loc[df['game_year'] == TEST_YEAR]
df_train = df.loc[df['game_year'] < TEST_YEAR]


NameError: name 'FEATURE_SET' is not defined

In [1691]:
df_test.head(5)


,Unnamed: 0,Rnd,R,Team,Opponent,Date,game_year,game_key,round_num,T,win,season_status,rolling_for,M,rolling_wins,rolling_games,rolling_against,rolling_margin,rolling_percentage,rolling_F_median_1,rolling_A_median_1,rolling_wins_1,rolling_wins_2,rolling_wins_3,prev_match_M_1,rolling_M_median_1,rolling_M_median_2,rolling_M_median_3,prev_match_M_2,prev_match_M_3,prev_match_M_4,prev_match_M_5,prev_match_M_6,prev_match_M_7,prev_match_M_8,prev_match_M_9,prev_match_M_10,prev_match_M_11,rolling_game_points,home_rolling_for,home_rolling_against,rolling_percentage_1,home_for,home_against,early,mid,game_points,F,A,home_game,home_win,away_game,away_win,winsum,games,percentage,percentage_wins,home_percentage_wins,away_percentage_wins,ladder_position,join_year,Rnd_joined,R_joined,Team_joined,Opponent_joined,game_year_joined,game_key_joined,T_joined,win_joined,season_status_joined,rolling_for_joined,M_joined,rolling_wins_joined,rolling_games_joined,rolling_against_joined,rolling_margin_joined,rolling_percentage_joined,rolling_F_median_1_joined,rolling_A_median_1_joined,rolling_wins_1_joined,rolling_wins_2_joined,rolling_wins_3_joined,prev_match_M_1_joined,rolling_M_median_1_joined,rolling_M_median_2_joined,rolling_M_median_3_joined,prev_match_M_2_joined,prev_match_M_3_joined,prev_match_M_4_joined,prev_match_M_5_joined,prev_match_M_6_joined,prev_match_M_7_joined,prev_match_M_8_joined,prev_match_M_9_joined,prev_match_M_10_joined,prev_match_M_11_joined,rolling_game_points_joined,home_rolling_for_joined,home_rolling_against_joined,rolling_percentage_1_joined,home_for_joined,home_against_joined,early_joined,mid_joined,game_points_joined,F_joined,A_joined,home_game_joined,home_win_joined,away_game_joined,away_win_joined,winsum_joined,games_joined,percentage_joined,percentage_wins_joined,home_percentage_wins_joined,away_percentage_wins_joined,ladder_position_joined,join_year_joined
301,605,R1,L,Adelaide,Fremantle,2022-03-20 15:40:00,2022,28186.02022-03-20 15:40:001,1,H,0,early,1596.0,-1,0,1,1,1595.0,1596.000000,61.0,83.5,4.0,4.0,0.0,1.0,-25.5,-10.0,-28.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,845.0,1,0.853707,82,83,1,0,28,1616,1971,11,5,11,2,7,22,0.819888,0.318182,0.454545,0.181818,15,2022,R1,W,Fremantle,Adelaide,2022,28186.02022-03-20 15:40:001,A,1,early,1548.0,1,0,1,1,1547.0,1548.000000,61.0,79.5,4.0,4.0,8.0,0.0,-6.0,12.0,-24.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0,793.0,1,0.605948,0,0,1,0,40,1578,1825,11,7,11,3,10,22,0.864658,0.454545,0.636364,0.272727,11,2022
302,607,R3,W,Adelaide,Port Adelaide,2022-04-01 19:50:00,2022,39190.02022-04-01 19:50:003,3,H,1,early,83.0,4,-43,3,184,-101.0,0.451087,58.5,91.0,4.0,0.0,4.0,0.0,-41.5,-42.0,-10.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,83.0,84,0.814371,96,92,1,0,28,1616,1971,11,5,11,2,7,22,0.819888,0.318182,0.454545,0.181818,15,2022,R3,L,Port Adelaide,Adelaide,2022,39190.02022-04-01 19:50:003,A,0,early,70.0,-4,-75,3,201,-131.0,0.348259,71.5,66.5,4.0,12.0,8.0,0.0,8.5,13.0,10.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0,57.0,121,1.122402,0,0,1,0,68,1884,1492,11,8,11,9,17,22,1.262735,0.772727,0.727273,0.818182,2,2022
303,609,R5,W,Adelaide,Richmond,2022-04-16 16:05:00,2022,33031.02022-04-16 16:05:005,5,H,1,early,278.0,19,-43,5,379,-101.0,0.733509,72.5,95.0,4.0,4.0,4.0,0.0,-4.0,-42.0,-10.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,4,179.0,176,0.925373,101,82,1,0,28,1616,1971,11,5,11,2,7,22,0.819888,0.318182,0.454545,0.181818,15,2022,R5,L,Richmond,Adelaide,2022,33031.02022-04-16 16:05:005,A,0,early,293.0,-19,16,5,353,-60.0,0.830028,83.5,84.5,8.0,2.0,4.0,1.0,-2.0,-4.0,-10.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,8,209.0,135,0.957486,0,0,1,0,38,1743,1780,11,6,11,3,9,22,0.979213,0.409091,0.545455,0.272727,12,2022
304,611,R7,L,Adelaide,Greater Western Sydney,2022-04-30 13:40:00,2022,31811.02022-04-30 13:40:007,7,H,0,mid,442.0,-59,-23,7,523,-81.0,0.845124,72.5,87.5,8.0,4.0,4.0,1.0,-2.5,-4.0,-10.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,12,280.0,258,0.955939,54,113,0,1,28,1616,1971,11,5,11,2,7,22,0.819888,0.3181

In [1692]:
settings = {
    "time_budget": .5*60,  # total running time in seconds
    "metric": "log_loss",  # can be: 'r2', 'rmse', 'mae', 'mse', 'accuracy', 'roc_auc', 'roc_auc_ovr',
    # 'roc_auc_ovo', 'log_loss', 'mape', 'f1', 'ap', 'ndcg', 'micro_f1', 'macro_f1'
    "task": "classification",  # task type
    "log_file_name": 'airlines_experiment.log',  # flaml log file
    "seed": 7654321,  # random seed
    "verbose": 0,
    "ensemble": True,
    # "sample": True,
    # "estimator_list": ["rf","lgbm"]
    "estimator_list": ["xgboost"],
    "eval_method": "cv",
    "n_splits": 10,
    # "split_type": "stratified"
}


In [1693]:


automl = AutoML()
automl.fit(df_train[TRAIN_COLUMNS], df_train[TARGET], **settings)


In [1694]:
print("Best ML leaner:", automl.best_estimator)
print("Best hyperparmeter config:", automl.best_config)
print("Best log_loss on validation data: {0:.4g}".format(automl.best_loss))
print("Training duration of best run: {0:.4g} s".format(automl.best_config_train_time))



Best ML leaner: xgboost
Best hyperparmeter config: {'n_estimators': 629, 'max_leaves': 14, 'min_child_weight': 10.270189876546125, 'learning_rate': 0.033969370647932366, 'subsample': 0.6721003635662315, 'colsample_bylevel': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 0.007737408313278222, 'reg_lambda': 1.7828509006028763}
Best log_loss on validation data: 0.6131
Training duration of best run: 0.2278 s


In [1695]:
automl.model
#

In [1696]:
"""compute predictions of testing dataset"""
# y_pred = automl.predict(X_test)
# print('Predicted labels', y_pred)
# print('True labels', y_test)
# df_score = df
df_test["score"] = automl.predict_proba(df_test[TRAIN_COLUMNS])[:, 1]


/var/folders/w7/y95z2by90x5368bylpv7ydt40000gn/T/ipykernel_69464/1815756750.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["score"] = automl.predict_proba(df_test[TRAIN_COLUMNS])[:, 1]


In [1697]:
def score_function(dataf):
    dataf.loc[dataf[TARGET] == 1, "prob_score"] = 1 + np.log2(dataf["score"])
    dataf.loc[dataf[TARGET] == 0, "prob_score"] = 1 + np.log2(1 - dataf["score"])
    return dataf["prob_score"]


In [1698]:
import numpy as np
# df_test['score'] = np.where(df_test['score']>=.88,.88,df_test['score'])
# df_test['score'] = np.where(df_test['score']<=.12,.12,df_test['score'])
df_test["prob_score"] = score_function(df_test)


/Users/scottgregory/miniconda3/envs/tapes/lib/python3.9/site-packages/pandas/core/indexing.py:1681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = empty_value
/Users/scottgregory/miniconda3/envs/tapes/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/var/folders/w7/y95z2by90x5368bylpv7ydt40000gn/T/ipykernel_69464/2779552605.py:4: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [1699]:
df_test[
    [
        "Date",
        "round_num",
        "Team",
        "Opponent",
        "score",
        "win",
        "prob_score",
        "rolling_for",
        "rolling_for_joined"
    ]
].sort_values(by="prob_score", ascending=True).head(30)

,Date,round_num,Team,Opponent,score,win,prob_score,rolling_for,rolling_for_joined
4764,2022-06-17 19:50:00,14,St Kilda,Essendon,0.890808,0,-2.195067,969.0,793.0
3741,2022-07-16 16:35:00,18,North Melbourne,Richmond,0.124448,1,-2.006380,872.0,1490.0
576,2022-07-10 13:10:00,17,Brisbane Lions,Essendon,0.849278,0,-1.730042,1447.0,1092.0
5143,2022-05-07 13:45:00,8,Sydney,Gold Coast,0.813265,0,-1.420935,625.0,500.0
1711,2022-07-02 13:45:00,16,Essendon,Sydney,0.230081,1,-1.119784,997.0,1200.0
3356,2022-06-04 19:25:00,12,Melbourne,Sydney,0.748775,0,-0.992949,939.0,984.0
4758,2022-03-18 19:50:00,1,St Kilda,Collingwood,0.746541,0,-0.980176,1612.0,1516.0
1996,2022-05-22 15:20:00,10,Fremantle,Collingwood,0.737035,0,-0.927058,704.0,753.0
1712,2022-07-17 16:40:00,18,Essendon,Gold Coast,0.264032,1,-0.921214,1192.0,1324.0
304,2022-04-30 13:40:00,7,Adelaide,Greater Western Sydney,0.734122,0,-0.911163,442.0,363.0


In [1700]:
# round_tester = df_test.loc[df_test['round_num']<=10]
# round_tester['prob_score'].sum()
df_test["prob_score"].sum()


37.297478

In [1701]:


# '''pickle and save the automl object'''
# import pickle
# with open('automl.pkl', 'wb') as f:
#     pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)
# '''load pickled automl object'''
# with open('automl.pkl', 'rb') as f:
#     automl = pickle.load(f)




In [1712]:
# current_round = pd.read_csv('current_round.csv')
current_round = df.loc[(df['game_year']==2023)&(df['round_num']==5)]

In [1713]:
current_round["score"] = automl.predict_proba(current_round[TRAIN_COLUMNS])[:, 1]
current_round[
    [
        "Date",
        "round_num",
        "Team",
        "Opponent",
        "score",
        "win",
        "rolling_for",
        "rolling_for_joined",
        "rolling_wins_1",
        "rolling_wins_1_joined"
    ]
].sort_values(by="score", ascending=True).head(20)

/var/folders/w7/y95z2by90x5368bylpv7ydt40000gn/T/ipykernel_69464/3331779147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_round["score"] = automl.predict_proba(current_round[TRAIN_COLUMNS])[:, 1]


,Date,round_num,Team,Opponent,score,win,rolling_for,rolling_for_joined,rolling_wins_1,rolling_wins_1_joined
11002,2023-12-31 00:00:00,5,West Coast,Geelong,0.304045,0,250.0,313.0,4.0,4.0
5006,2023-12-31 00:00:00,5,Gold Coast,Fremantle,0.336603,0,215.0,253.0,4.0,4.0
8793,2023-12-31 00:00:00,5,Richmond,Sydney,0.460603,0,251.0,313.0,4.0,4.0
3437,2023-12-31 00:00:00,5,Essendon,Melbourne,0.469641,0,321.0,376.0,8.0,8.0
7501,2023-12-31 00:00:00,5,North Melbourne,Brisbane Lions,0.472779,0,245.0,282.0,4.0,8.0
2680,2023-12-31 00:00:00,5,Collingwood,St Kilda,0.506015,0,344.0,298.0,8.0,12.0
5980,2023-12-31 00:00:00,5,Hawthorn,Greater Western Sydney,0.508223,0,191.0,263.0,4.0,0.0
8037,2023-12-31 00:00:00,5,Port Adelaide,Western Bulldogs,0.524894,0,279.0,222.0,4.0,8.0
11777,2023-12-31 00:00:00,5,Western Bulldogs,Port Adelaide,0.548547,0,222.0,279.0,8.0,4.0
631,2023-12-31 00:00:00,5,Adelaide,Carlton,0.590277,0,319.0,272.0,8.0,12.0


In [1723]:
# home = current_round[['Team','score']]
# away = current_round[['Opponent','score']]
# away['score'] = 1-away['score']
# away.columns = ['Team','score']
# home.append(away).sort_values(by='Team').groupby('Team').mean()

In [1724]:
current_round["score"] = automl.predict_proba(current_round[TRAIN_COLUMNS])[:, 1]
current_round[
    [
        "Date",
        "round_num",
        "Team",
        "Opponent",
        "score",
        "win",
        "rolling_for",
        "rolling_for_joined",
        "rolling_wins_1",
        "rolling_wins_1_joined"
    ]
].sort_values(by="score", ascending=True).head(20)

/var/folders/w7/y95z2by90x5368bylpv7ydt40000gn/T/ipykernel_69464/3331779147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_round["score"] = automl.predict_proba(current_round[TRAIN_COLUMNS])[:, 1]


,Date,round_num,Team,Opponent,score,win,rolling_for,rolling_for_joined,rolling_wins_1,rolling_wins_1_joined
11002,2023-12-31 00:00:00,5,West Coast,Geelong,0.304045,0,250.0,313.0,4.0,4.0
5006,2023-12-31 00:00:00,5,Gold Coast,Fremantle,0.336603,0,215.0,253.0,4.0,4.0
8793,2023-12-31 00:00:00,5,Richmond,Sydney,0.460603,0,251.0,313.0,4.0,4.0
3437,2023-12-31 00:00:00,5,Essendon,Melbourne,0.469641,0,321.0,376.0,8.0,8.0
7501,2023-12-31 00:00:00,5,North Melbourne,Brisbane Lions,0.472779,0,245.0,282.0,4.0,8.0
2680,2023-12-31 00:00:00,5,Collingwood,St Kilda,0.506015,0,344.0,298.0,8.0,12.0
5980,2023-12-31 00:00:00,5,Hawthorn,Greater Western Sydney,0.508223,0,191.0,263.0,4.0,0.0
8037,2023-12-31 00:00:00,5,Port Adelaide,Western Bulldogs,0.524894,0,279.0,222.0,4.0,8.0
11777,2023-12-31 00:00:00,5,Western Bulldogs,Port Adelaide,0.548547,0,222.0,279.0,8.0,4.0
631,2023-12-31 00:00:00,5,Adelaide,Carlton,0.590277,0,319.0,272.0,8.0,12.0


In [1708]:
mapper = pd.read_csv('submit_mapper.csv')
df_mapped = pd.merge(current_round, mapper, left_on='Team', right_on='Season_team')

In [1709]:
df_mapped

,Unnamed: 0,Rnd,R,Team,Opponent,Date,game_year,game_key,round_num,T,win,season_status,rolling_for,M,rolling_wins,rolling_games,rolling_against,rolling_margin,rolling_percentage,rolling_F_median_1,rolling_A_median_1,rolling_wins_1,rolling_wins_2,rolling_wins_3,prev_match_M_1,rolling_M_median_1,rolling_M_median_2,rolling_M_median_3,prev_match_M_2,prev_match_M_3,prev_match_M_4,prev_match_M_5,prev_match_M_6,prev_match_M_7,prev_match_M_8,prev_match_M_9,prev_match_M_10,prev_match_M_11,rolling_game_points,home_rolling_for,home_rolling_against,rolling_percentage_1,home_for,home_against,early,mid,game_points,F,A,home_game,home_win,away_game,away_win,winsum,games,percentage,percentage_wins,home_percentage_wins,away_percentage_wins,ladder_position,join_year,Rnd_joined,R_joined,Team_joined,Opponent_joined,game_year_joined,game_key_joined,T_joined,win_joined,season_status_joined,rolling_for_joined,M_joined,rolling_wins_joined,rolling_games_joined,rolling_against_joined,rolling_margin_joined,rolling_percentage_joined,rolling_F_median_1_joined,rolling_A_median_1_joined,rolling_wins_1_joined,rolling_wins_2_joined,rolling_wins_3_joined,prev_match_M_1_joined,rolling_M_median_1_joined,rolling_M_median_2_joined,rolling_M_median_3_joined,prev_match_M_2_joined,prev_match_M_3_joined,prev_match_M_4_joined,prev_match_M_5_joined,prev_match_M_6_joined,prev_match_M_7_joined,prev_match_M_8_joined,prev_match_M_9_joined,prev_match_M_10_joined,prev_match_M_11_joined,rolling_game_points_joined,home_rolling_for_joined,home_rolling_against_joined,rolling_percentage_1_joined,home_for_joined,home_against_joined,early_joined,mid_joined,game_points_joined,F_joined,A_joined,home_game_joined,home_win_joined,away_game_joined,away_win_joined,winsum_joined,games_joined,percentage_joined,percentage_wins_joined,home_percentage_wins_joined,away_percentage_wins_joined,ladder_position_joined,join_year_joined,score,Submit_team,Season_team
0,631,R5,U,Adelaide,Carlton,2023-12-31 00:00:00,2023,0.02023-12-31 00:00:005,5,H,0,early,319.0,0,22,5,373,-54.0,0.855228,88.0,88.5,8.0,4.0,8.0,1.0,5.5,-5.0,-29.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,8,188.0,181,0.991023,0,0,1,0,32,1721,1986,11,5,11,3,8,22,0.866566,0.363636,0.454545,0.272727,14,2023,R5,U,Carlton,Adelaide,2023,0.02023-12-31 00:00:005,A,0,early,272.0,0,41,5,289,-17.0,0.941176,74.0,80.5,12.0,2.0,4.0,1.0,-0.5,-29.0,15.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,14,91.0,83,1.079186,0,0,1,0,48,1857,1714,11,8,11,4,12,22,1.083431,0.545455,0.727273,0.363636,9,2023,0.590277,Adelaide,Adelaide
1,5006,R5,U,Gold Coast,Fremantle,2023-12-31 00:00:00,2023,0.02023-12-31 00:00:005,5,H,0,early,215.0,0,-111,5,386,-171.0,0.556995,68.0,106.0,4.0,4.0,4.0,0.0,-22.5,-7.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,4,135.0,165,0.811252,0,0,1,0,40,1871,1820,11,7,11,3,10,22,1.028022,0.454545,0.636364,0.272727,12,2023,R5,U,Fremantle,Gold Coast,2023,0.02023-12-31 00:00:005,A,0,early,253.0,0,-14,5,319,-66.0,0.793103,71.5,71.0,4.0,8.0,6.0,0.0,-0.5,0.0,13.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,4,181.0,141,1.069124,0,0,1,0,62,1739,1486,11,7,11,8,15,22,1.170256,0.681818,0.636364,0.727273,5,2023,0.336603,Gold_Coast,Gold Coast
2,5226,R5,U,Greater Western Sydney,Hawthorn,2023-12-31 00:00:00,2023,0.02023-12-31 00:00:005,5,H,0,early,263.0,0,-26,5,353,-90.0,0.745042,66.5,89.5,0.0,4.0,4.0,0.0,-16.0,-40.0,-11.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,171.0,165,0.898608,0,0,1,0,24,1631,1927,11,4,11,2,6,22,0.846393,0.272727,0.363636,0.181818,16,2023,R5,U,Hawthorn,Greater Western Sydney,2023,0.02023-12-31 00:00:005,A,0,early,191.0,0,-203,5,431,-240.0,0.443155,66.0,82.0,4.0,0.0,8.0,0.0,-17.5,25.0,-13.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,4,146.0,186,0.555039,0,0,1,0,32,1787,1991,11,6,11,2,8,22,0.897539,0.363636,0.545455,0.181818,13,2023,0.648003,G_W_Sydney,Greater Western Sydney
3,6736,R5,U,Melbourne,Essendon,2023-12-31 00:00:00,2023,0.02023-12-31 00:00:005,5,H,0,early,376.0,0,152,5,306,70.0,1.228758,94.5,71.5,8.0,12.0,4.0,1.0,30.0,-7.0,-12.0,1.0,0.0,1.0,1.0,1.0,

In [1710]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
browser = webdriver.Firefox()

browser.get("http://probabilistic-footy.monash.edu/~footy/tips.shtml")

time.sleep(3)
# username = browser.find_element_by_xpath("/html/body/form/table/tbody/tr[1]/td[2]/input")
# password = browser.find_element_by_xpath("/html/body/form/table/tbody/tr[2]/td[2]/input")
username = browser.find_element('name','name')
password = browser.find_element('name','passwd')
username.send_keys("")
password.send_keys("")
login_attempt = browser.find_element("xpath","//*[@type='submit']")
login_attempt.submit()

time.sleep(3)
main_table = browser.find_elements(By.TAG_NAME,'tbody')
rower = main_table[1].find_elements(By.TAG_NAME,'tr')
for rows in range(len(rower)-1):
    home_team = rower[rows+1].find_elements(By.TAG_NAME,'td')[2].text
    try:
        prediction = df_mapped.loc[df_mapped['Submit_team'] == home_team]['score'].values[0]
        #gamer = rower[rows+1].find_elements_by_tag_name('td')[4]
        gamer = browser.find_element('name','game'+str(rows+1))
        gamer.clear()
        gamer.send_keys(prediction.astype('str'))
    except:
        print('Game not found')
   # print('game'+str(rows+1))
   # print(prediction)

time.sleep(3)
login_attempt = browser.find_element("xpath","//*[@type='submit']")
# login_attempt.submit()


Game not found
Game not found
Game not found
Game not found
Game not found
Game not found


In [37]:
import datetime
def round_adder(round_num):
    tabler = pd.read_html("https://en.wikipedia.org/wiki/2023_AFL_season")
    tt = tabler[round_num+3]
    print(tt)
    # if True:
    if (len(tt) == 10) | (len(tt) == 12):
        rounder = "R" + str(round_num)
        tt = tt.loc[tt[2] == "vs."]
        tt["Rnd"] = rounder
        tt["T"] = "H"
        tt["Opponent"] = tt[3]
        tt["F"] = 0
        tt["A"] = 0
        tt["R"] = "U"
        tt["M"] = 0
        tt["W-D-L"] = "U"
        tt["Venue"] = tt[4]
        tt["Crowd"] = 0
        tt["Date"] = datetime.datetime(2023, 12, 31)
        tt["Team"] = tt[1]
        cols = [
            "Rnd",
            "T",
            "Opponent",
            "F",
            "A",
            "R",
            "M",
            "W-D-L",
            "Venue",
            "Crowd",
            "Date",
            "Team",
        ]
        table_out = tt[cols]
        aa = table_out.copy()
        aa["opp_stage"] = aa["Opponent"]
        aa["Opponent"] = aa["Team"]
        aa["Team"] = aa["opp_stage"]
        aa["T"] = "A"
        aa = aa[cols]
        table_out = pd.concat([table_out, aa])
    return table_out


In [38]:
round_adder(7).head(5)

                               0                            1        2  \
0                            NaN                          NaN      NaN   
1                            NaN                      Round 7  Round 7   
2     Friday, 28 April (7:50 pm)          St Kilda 11.10 (76)  def. by   
3   Saturday, 29 April (1:45 pm)   Brisbane Lions 17.13 (115)     def.   
4   Saturday, 29 April (2:10 pm)           Sydney 16.10 (106)  def. by   
5   Saturday, 29 April (4:35 pm)  Western Bulldogs 14.10 (94)     def.   
6   Saturday, 29 April (7:25 pm)         Melbourne 22.7 (139)     def.   
7   Saturday, 29 April (5:30 pm)          West Coast 6.8 (44)  def. by   
8     Sunday, 30 April (1:10 pm)          Essendon 16.8 (104)  def. by   
9     Sunday, 30 April (3:20 pm)           Richmond 6.12 (48)  def. by   
10    Sunday, 30 April (4:10 pm)           Adelaide 7.16 (58)  def. by   
11                           NaN                          NaN      NaN   

                                    3

,Rnd,T,Opponent,F,A,R,M,W-D-L,Venue,Crowd,Date,Team


In [458]:
x = round_adder(1)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
browser = webdriver.Firefox()

browser.get("http://probabilistic-footy.monash.edu/~footy/tips.shtml")

time.sleep(3)
# username = browser.find_element_by_xpath("/html/body/form/table/tbody/tr[1]/td[2]/input")
# password = browser.find_element_by_xpath("/html/body/form/table/tbody/tr[2]/td[2]/input")
username = browser.find_element('name','name')
password = browser.find_element('name','passwd')
username.send_keys("")
password.send_keys("")
login_attempt = browser.find_element("xpath","//*[@type='submit']")
login_attempt.submit()

time.sleep(3)
main_table = browser.find_elements(By.TAG_NAME,'tbody')
rower = main_table[1].find_elements(By.TAG_NAME,'tr')
for rows in range(len(rower)-1):
    home_team = rower[rows+1].find_elements(By.TAG_NAME,'td')[2].text
    try:
        prediction = df_mapped.loc[df_mapped['Submit_team'] == home_team]['score'].values[0]
        #gamer = rower[rows+1].find_elements_by_tag_name('td')[4]
        gamer = browser.find_element('name','game'+str(rows+1))
        gamer.clear()
        gamer.send_keys(prediction.astype('str'))
    except:
        print('Game not found')
   # print('game'+str(rows+1))
   # print(prediction)

time.sleep(3)
login_attempt = browser.find_element("xpath","//*[@type='submit']")
# login_attempt.submit()


In [736]:
x = [84,134,118]

In [737]:
np.quantile(x,.95,method='lower')

118

In [738]:
np.quantile(x,.1,method='higher')

118

In [732]:
df.loc[df['Team']=='Port Adelaide'].sort_values(by='Date',ascending=False).head(5)

,Unnamed: 0,Rnd,R,Team,Opponent,Date,game_year,game_key,round_num,T,win,season_status,rolling_for,M,rolling_wins,rolling_games,rolling_against,rolling_margin,rolling_percentage,rolling_wins_1,rolling_wins_2,rolling_wins_3,prev_match_M_1,rolling_game_points,home_rolling_for,home_rolling_against,rolling_percentage_1,home_for,home_against,early,mid,game_points,F,A,home_game,home_win,away_game,away_win,winsum,games,percentage,percentage_wins,home_percentage_wins,away_percentage_wins,ladder_position,join_year,Rnd_joined,R_joined,Team_joined,Opponent_joined,game_year_joined,game_key_joined,T_joined,win_joined,season_status_joined,rolling_for_joined,M_joined,rolling_wins_joined,rolling_games_joined,rolling_against_joined,rolling_margin_joined,rolling_percentage_joined,rolling_wins_1_joined,rolling_wins_2_joined,rolling_wins_3_joined,prev_match_M_1_joined,rolling_game_points_joined,home_rolling_for_joined,home_rolling_against_joined,rolling_percentage_1_joined,home_for_joined,home_against_joined,early_joined,mid_joined,game_points_joined,F_joined,A_joined,home_game_joined,home_win_joined,away_game_joined,away_win_joined,winsum_joined,games_joined,percentage_joined,percentage_wins_joined,home_percentage_wins_joined,away_percentage_wins_joined,ladder_position_joined,join_year_joined
2361,4723,R3,L,Port Adelaide,Adelaide,2023-04-01 19:00:00,2023,48962.02023-04-01 19:00:003,3,H,0,early,191,-31,1,3,208,-17,0.918269,12.0,4.0,4.0,85.0,4,127,73,1.379630,86,117,1,0,40,1806,1638,11,7,11,3,10,22,1.102564,0.454545,0.636364,0.272727,11,2023,R3,W,Adelaide,Port Adelaide,2023,48962.02023-04-01 19:00:003,A,1,early,167,31,0,3,215,-48,0.776744,8.0,4.0,4.0,-59.0,0,77,109,0.812030,0,0,1,0,32,1721,1986,11,5,11,3,8,22,0.866566,0.363636,0.454545,0.272727,14,2023
2360,4721,R1,W,Port Adelaide,Brisbane Lions,2023-03-18 16:05:00,2023,34255.02023-03-18 16:05:001,1,H,1,early,1,54,0,1,1,0,1.000000,8.0,8.0,4.0,84.0,0,1,1,1.305732,126,72,1,0,40,1806,1638,11,7,11,3,10,22,1.102564,0.454545,0.636364,0.272727,11,2023,R1,L,Brisbane Lions,Port Adelaide,2023,34255.02023-03-18 16:05:001,A,0,early,1,-54,0,1,1,0,1.000000,12.0,8.0,4.0,0.0,0,1,1,0.953552,0,0,1,0,60,2147,1799,11,9,11,6,15,22,1.193441,0.681818,0.818182,0.545455,6,2023
2359,4720,R23,W,Port Adelaide,Adelaide,2022-08-20 19:00:00,2022,50090.02022-08-20 19:00:0023,23,H,1,late,1696,56,9,22,1584,112,1.070707,4.0,12.0,4.0,14.0,36,786,735,1.076712,111,55,0,0,68,1884,1492,11,8,11,9,17,22,1.262735,0.772727,0.727273,0.818182,2,2022,R23,L,Adelaide,Port Adelaide,2022,50090.02022-08-20 19:00:0023,A,0,late,1667,-56,8,22,1876,-209,0.888593,12.0,4.0,4.0,36.0,32,895,934,1.123123,0,0,0,0,28,1616,1971,11,5,11,2,7,22,0.819888,0.318182,0.454545,0.181818,15,2022
2358,4718,R21,L,Port Adelaide,Richmond,2022-08-06 19:10:00,2022,27091.02022-08-06 19:10:0021,21,H,0,late,1479,-38,8,20,1413,66,1.046709,4.0,12.0,4.0,15.0,32,715,626,1.075163,71,109,0,0,68,1884,1492,11,8,11,9,17,22,1.262735,0.772727,0.727273,0.818182,2,2022,R21,W,Richmond,Port Adelaide,2022,27091.02022-08-06 19:10:0021,A,1,late,1788,38,10,20,1568,220,1.140306,10.0,8.0,8.0,36.0,42,898,718,1.002985,0,0,0,0,38,1743,1780,11,6,11,3,9,22,0.979213,0.409091,0.545455,0.272727,12,2022
2357,4716,R19,L,Port Adelaide,Geelong,2022-07-23 16:05:00,2022,30937.02022-07-23 16:05:0019,19,H,0,late,1303,-12,8,18,1219,84,1.068909,12.0,8.0,8.0,58.0,32,621,520,1.115894,94,106,0,0,68,1884,1492,11,8,11,9,17,22,1.262735,0.772727,0.727273,0.818182,2,2022,R19,W,Geelong,Port Adelaide,2022,30937.02022-07-23 16:05:0019,A,1,late,1587,12,13,18,1159,428,1.369284,20.0,12.0,4.0,191.0,52,788,495,1.733051,0,0,0,0,64,1845,1456,11,9,11,7,16,22,1.267170,0.727273,0.818182,0.636364,3,2022


In [1727]:
import os

print(os.getcwd())
print(sys.argv[0])
print(os.path.dirname(os.path.realpath('__file__')))

/Users/scottgregory/Documents/gits/mlflow_afl
/Users/scottgregory/miniconda3/envs/tapes/lib/python3.9/site-packages/ipykernel_launcher.py
/Users/scottgregory/Documents/gits/mlflow_afl


In [1728]:
os.environ

environ{'NVM_INC': '/Users/scottgregory/.nvm/versions/node/v18.6.0/include/node',
        'VSCODE_CLI': '1',
        'CONDA_BACKUP_DEBUG_CXXFLAGS': '-ftree-vectorize -fPIC -fPIE -fstack-protector-strong -O2 -pipe -stdlib=libc++ -fvisibility-inlines-hidden -fmessage-length=0 -Og -g -Wall -Wextra -isystem /Users/scottgregory/miniconda3/envs/ThinkBayes2/include',
        'CONDA_BACKUP_RANLIB': 'arm64-apple-darwin20.0.0-ranlib',
        'MANPATH': '/Users/scottgregory/.nvm/versions/node/v18.6.0/share/man:/opt/homebrew/share/man::',
        'CONDA_BACKUP_OBJC_FOR_BUILD': '/Users/scottgregory/miniconda3/envs/ThinkBayes2/bin/@OBJC_FOR_BUILD@',
        'TERM_PROGRAM': 'iTerm.app',
        'NVM_CD_FLAGS': '-q',
        'CONDA_BACKUP_INSTALL_NAME_TOOL': 'arm64-apple-darwin20.0.0-install_name_tool',
        'CONDA_BACKUP_REDO_PREBINDING': 'arm64-apple-darwin20.0.0-redo_prebinding',
        'TERM': 'xterm-color',
        'HOST': 'Scotts-Mac-mini.modem',
        'SHELL': '/bin/zsh',
        'HOMEBR

In [1731]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.environ["ROOT_DIRECTORY"])

/Users/scottgregory/Documents/gits/mlflow_afl


In [1746]:
x = pd.read_csv('data/raw/games.csv')

In [8]:
import pandas as pd
x = pd.read_csv('/Users/scottgregory/Documents/gits/mlflow_afl/data/scored/scored_2023_7_automl_lgbm_500_nf.csv')

In [9]:
x

,Submit_team,Opponent,score,rolling_for,rolling_for_joined,rolling_against,rolling_against_joined,rolling_margin,rolling_margin_joined,rolling_percentage,rolling_percentage_joined,rolling_wins_1,rolling_wins_1_joined,rolling_wins_2,rolling_wins_2_joined,rolling_wins_3,rolling_wins_3_joined,rolling_F_median_1,rolling_F_median_1_joined,rolling_A_median_1,rolling_A_median_1_joined,rolling_M_median_1,rolling_M_median_1_joined,rolling_F_sum_1,rolling_F_sum_1_joined,rolling_A_sum_1,rolling_A_sum_1_joined,rolling_M_sum_1,rolling_M_sum_1_joined,rolling_M_1,rolling_M_1_joined,rolling_M_2,rolling_M_2_joined,rolling_M_3,rolling_M_3_joined,round_num,early,mid
0,Adelaide,Collingwood,0.542883,516.0,504.0,511,474,5.0,30.0,1.009785,1.063291,12.0,8.0,4.0,12.0,8.0,8.0,96.0,85.5,81.0,77.0,22.5,6.5,935.0,875.0,836.0,795.0,99.0,80.0,98.0,-14.0,69.0,85.0,34.646729,22.033308,7,0,1
1,Brisbane,Fremantle,0.626798,542.0,422.0,523,527,19.0,-105.0,1.036329,0.800759,12.0,4.0,4.0,4.0,8.0,12.0,95.0,72.0,82.5,75.5,13.0,4.5,943.0,767.0,888.0,805.0,55.0,-38.0,129.0,-78.0,60.0,-57.0,38.722033,28.541709,7,0,1
2,Essendon,Geelong,0.356239,502.0,579.0,480,460,22.0,119.0,1.045833,1.258696,8.0,12.0,8.0,0.0,0.0,12.0,82.5,114.5,85.0,65.5,0.0,46.0,895.0,1086.0,928.0,695.0,-33.0,391.0,27.0,222.0,88.0,161.0,38.881872,41.448764,7,0,1
3,Melbourne,North Melbourne,0.900314,554.0,383.0,488,591,66.0,-208.0,1.135246,0.648054,8.0,0.0,8.0,8.0,8.0,0.0,92.5,73.5,76.0,105.0,32.0,-33.5,998.0,711.0,753.0,1047.0,245.0,-336.0,54.0,-141.0,135.0,-154.0,32.199120,32.923902,7,0,1
4,Richmond,Gold Coast,0.724009,407.0,402.0,505,540,-98.0,-138.0,0.805941,0.744444,0.0,4.0,6.0,4.0,12.0,4.0,94.0,76.5,75.5,102.0,3.5,-8.5,937.0,804.0,814.0,879.0,123.0,-75.0,-67.0,-20.0,-49.0,-82.0,29.372819,34.412147,7,0,1
5,St_Kilda,Port Adelaide,0.714626,447.0,458.0,358,514,89.0,-56.0,1.248603,0.891051,8.0,12.0,12.0,4.0,0.0,8.0,74.5,84.0,66.5,70.5,13.5,8.0,790.0,931.0,699.0,827.0,91.0,104.0,69.0,56.0,145.0,-4.0,31.761350,40.673445,7,0,1
6,Sydney,Greater Western Sydney,0.638199,499.0,427.0,507,536,-8.0,-109.0,0.984221,0.796642,4.0,4.0,8.0,4.0,12.0,4.0,99.0,76.0,70.0,88.5,32.5,-11.5,938.0,751.0,757.0,867.0,181.0,-116.0,-51.0,-32.0,25.0,-49.0,44.703281,26.906009,7,0,1
7,W_Coast,Carlton,0.351689,408.0,394.0,648,489,-240.0,-95.0,0.629630,0.805726,0.0,4.0,4.0,10.0,0.0,0.0,75.5,74.0,107.5,82.0,-32.0,-3.0,753.0,735.0,1058.0,840.0,-305.0,-105.0,-150.0,-55.0,-177.0,-41.0,28.497661,30.105371,7,0,1
8,W_Bulldogs,Hawthorn,0.723272,396.0,342.0,484,587,-88.0,-245.0,0.818182,0.582624,8.0,0.0,4.0,4.0,8.0,4.0,66.5,66.0,77.0,83.0,-4.5,-17.5,729.0,642.0,793.0,939.0,-64.0,-297.0,40.0,-87.0,-55.0,-208.0,32.968419,39.526995,7,0,1


import pandas as pd

x = [1,2,3]
y = [3,4,5]


In [8]:
import pandas as pd

x = [1,2,3]
y = [3,4,5]

dft = pd.DataFrame({'score':x,'target':y})
dft


,score,target
0,1,3
1,2,4
2,3,5


In [9]:
.1155*9

1.0395

In [10]:
from dotenv import load_dotenv
import os

load_dotenv()
ROOT_DIRECTORY = os.environ["ROOT_DIRECTORY"]

pd.options.mode.chained_assignment = None  # default='warn'

df = pd.read_csv(ROOT_DIRECTORY + "/data/raw/games.csv")

In [56]:
# df["game_year"] = pd.DatetimeIndex(df["Date"]).year
xd = df.groupby(['Team'])[['game_year']].min().reset_index()
xd['rem_flag'] = 1

In [57]:
xd

,Team,game_year,rem_flag
0,Adelaide,1993,1
1,Brisbane Lions,1999,1
2,Carlton,1986,1
3,Collingwood,1986,1
4,Essendon,1986,1
5,Fremantle,1997,1
6,Geelong,1986,1
7,Gold Coast,2013,1
8,Greater Western Sydney,2014,1
9,Hawthorn,1986,1


In [51]:
xy = pd.merge(left=df, right=xd, on=['Team','game_year'], how='left', suffixes=['','rem_'])

In [52]:
xy.loc[xy['rem_flag'].isna()]

,Unnamed: 0,Rnd,R,Team,Opponent,Date,game_year,game_key,round_num,T,...,winsum_joined,games_joined,percentage_joined,percentage_wins_joined,home_percentage_wins_joined,away_percentage_wins_joined,ladder_position_joined,join_year_joined,rem_flag,rem_flagrem_
0,0,R5,W,Adelaide,Richmond,1992-04-19 13:40:00,1992,39022.01992-04-19 13:40:005,5,H,...,7,22,0.873878,0.318182,0.272727,0.363636,13,1992,NaN,1.0
1,1,R9,L,Adelaide,Collingwood,1992-05-17 17:00:00,1992,44723.01992-05-17 17:00:009,9,H,...,12,22,1.155435,0.545455,0.636364,0.454545,7,1992,NaN,1.0
2,2,R11,W,Adelaide,North Melbourne,1992-05-31 13:40:00,1992,36312.01992-05-31 13:40:0011,11,H,...,12,22,0.911994,0.545455,0.636364,0.454545,8,1992,NaN,1.0
3,3,R14,L,Adelaide,Essendon,1992-06-21 17:15:00,1992,43163.01992-06-21 17:15:0014,14,H,...,13,22,1.092216,0.590909,0.636364,0.545455,6,1992,NaN,1.0
4,4,R15,W,Adelaide,Sydney,1992-06-26 20:10:00,1992,31593.01992-06-26 20:10:0015,15,H,...,7,22,0.849532,0.318182,0.363636,0.272727,12,1992,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5893,5893,R19,W,Western Bulldogs,Melbourne,2022-07-23 19:25:00,2022,26501.02022-07-23 19:25:0019,19,H,...,17,22,1.308385,0.772727,0.636364,0.909091,1,2022,NaN,NaN
5894,5894,R21,L,Western Bulldogs,Fremantle,2022-08-06 16:35:00,2022,24130.02022-08-06 16:35:0021,21,H,...,10,22,0.864658,0.454545,0.636364,0.272727,11,2022,NaN,NaN
5895,5895,R22,W,Western Bulldogs,Greater Western Sydney,2022-08-13 13:45:00,2022,18301.02022-08-13 13:45:0022,22,H,...,11,22,0.997180,0.500000,0.363636,0.636364,7,2022,NaN,NaN
5896,5896,R2,L,Western Bulldogs,St Kilda,2023-03-25 19:25:00,2023,33294.02023-03-25 19:25:002,2,H,...,11,22,0.993003,0.500000,0.454545,0.545455,10,2023,NaN,NaN


In [54]:
df = pd.read_csv(ROOT_DIRECTORY + "/data/processed/processed_features.csv")

In [55]:
len(df)

5730

In [51]:
def round_adder(round_num):
    tabler = pd.read_html("https://en.wikipedia.org/wiki/2023_AFL_season")
    tt = tabler[round_num + 3]
    print(tt)
    print(len(tt))
    if (len(tt) == 10) | (len(tt) == 12):
        rounder = "R" + str(round_num)
        tt = tt.loc[tt[2] == "v"]
        print(tt)
        tt["Rnd"] = rounder
        tt["T"] = "H"
        tt["Opponent"] = tt[3]
        tt["F"] = 0
        tt["A"] = 0
        tt["R"] = "U"
        tt["M"] = 0
        tt["W-D-L"] = "U"
        tt["Venue"] = tt[4]
        tt["Crowd"] = 0
        tt["Date"] = datetime.datetime(2023, 12, 31)
        tt["Team"] = tt[1]
        cols = [
            "Rnd",
            "T",
            "Opponent",
            "F",
            "A",
            "R",
            "M",
            "W-D-L",
            "Venue",
            "Crowd",
            "Date",
            "Team",
        ]
        table_out = tt[cols]
        aa = table_out.copy()
        aa["opp_stage"] = aa["Opponent"]
        aa["Opponent"] = aa["Team"]
        aa["Team"] = aa["opp_stage"]
        aa["T"] = "A"
        aa = aa[cols]
        table_out = pd.concat([table_out, aa])
    return table_out

In [52]:
x = round_adder(8)

                            0                       1        2  \
0                         NaN                     NaN      NaN   
1                         NaN                 Round 8  Round 8   
2     Friday, 5 May (7:50 pm)                 Carlton        v   
3   Saturday, 6 May (1:45 pm)                Richmond        v   
4   Saturday, 6 May (2:10 pm)                 Geelong        v   
5   Saturday, 6 May (4:35 pm)              Gold Coast        v   
6   Saturday, 6 May (7:25 pm)  Greater Western Sydney        v   
7   Saturday, 6 May (5:30 pm)               Fremantle        v   
8    Sunday, 7 May (12:40 pm)           Port Adelaide        v   
9     Sunday, 7 May (3:20 pm)             Collingwood        v   
10    Sunday, 7 May (4:40 pm)         North Melbourne        v   
11                        NaN                     NaN      NaN   

                   3                         4   5  
0                NaN                       NaN NaN  
1            Round 8               

/var/folders/w7/y95z2by90x5368bylpv7ydt40000gn/T/ipykernel_20531/757925271.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tt["Rnd"] = rounder
/var/folders/w7/y95z2by90x5368bylpv7ydt40000gn/T/ipykernel_20531/757925271.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tt["T"] = "H"
/var/folders/w7/y95z2by90x5368bylpv7ydt40000gn/T/ipykernel_20531/757925271.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [53]:
x

,Rnd,T,Opponent,F,A,R,M,W-D-L,Venue,Crowd,Date,Team
2,R8,H,Brisbane Lions,0,0,U,0,U,Marvel Stadium,0,2023-12-31,Carlton
3,R8,H,West Coast,0,0,U,0,U,Melbourne Cricket Ground,0,2023-12-31,Richmond
4,R8,H,Adelaide,0,0,U,0,U,GMHBA Stadium,0,2023-12-31,Geelong
5,R8,H,Melbourne,0,0,U,0,U,Heritage Bank Stadium,0,2023-12-31,Gold Coast
6,R8,H,Western Bulldogs,0,0,U,0,U,Manuka Oval,0,2023-12-31,Greater Western Sydney
7,R8,H,Hawthorn,0,0,U,0,U,Optus Stadium,0,2023-12-31,Fremantle
8,R8,H,Essendon,0,0,U,0,U,Adelaide Oval,0,2023-12-31,Port Adelaide
9,R8,H,Sydney,0,0,U,0,U,Melbourne Cricket Ground,0,2023-12-31,Collingwood
10,R8,H,St Kilda,0,0,U,0,U,Marvel Stadium,0,2023-12-31,North Melbourne
2,R8,A,Carlton,0,0,U,0,U,Marvel Stadium,0,2023-12-31,Brisbane Lions


In [57]:
x

,Rnd,T,Opponent,F,A,R,M,W-D-L,Venue,Crowd,Date,Team
2,R8,H,Brisbane Lions,0,0,U,0,U,Marvel Stadium,0,2023-12-31,Carlton
3,R8,H,West Coast,0,0,U,0,U,Melbourne Cricket Ground,0,2023-12-31,Richmond
4,R8,H,Adelaide,0,0,U,0,U,GMHBA Stadium,0,2023-12-31,Geelong
5,R8,H,Melbourne,0,0,U,0,U,Heritage Bank Stadium,0,2023-12-31,Gold Coast
6,R8,H,Western Bulldogs,0,0,U,0,U,Manuka Oval,0,2023-12-31,Greater Western Sydney
7,R8,H,Hawthorn,0,0,U,0,U,Optus Stadium,0,2023-12-31,Fremantle
8,R8,H,Essendon,0,0,U,0,U,Adelaide Oval,0,2023-12-31,Port Adelaide
9,R8,H,Sydney,0,0,U,0,U,Melbourne Cricket Ground,0,2023-12-31,Collingwood
10,R8,H,St Kilda,0,0,U,0,U,Marvel Stadium,0,2023-12-31,North Melbourne
2,R8,A,Carlton,0,0,U,0,U,Marvel Stadium,0,2023-12-31,Brisbane Lions
